## MODEL TRAINING

#### IMPORT LIBRARY

In [25]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

#### Data save model

In [26]:
data_dir = r'C:\machine learning\plate-detection\data'

batch_size = 32
img_height = 40
img_width = 40

#### preprocessing

In [27]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names

Found 360 files belonging to 36 classes.
Using 288 files for training.
Found 360 files belonging to 36 classes.
Using 72 files for validation.


#### Model result

In [28]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

num_classes = 36

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_9 (Rescaling)     (None, 40, 40, 3)         0         
                                                                 
 conv2d_12 (Conv2D)          (None, 40, 40, 16)        448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 20, 20, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 20, 20, 32)        4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 10, 10, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 10, 10, 64)       

#### Training model

In [29]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

model.save('model') 

Epoch 1/10
9/9 [==============================] - 11s 102ms/step - loss: 3.5591 - accuracy: 0.0521 - val_loss: 3.4369 - val_accuracy: 0.1667
Epoch 2/10
9/9 [==============================] - 0s 13ms/step - loss: 3.2003 - accuracy: 0.2778 - val_loss: 2.8395 - val_accuracy: 0.3611
Epoch 3/10
9/9 [==============================] - 0s 11ms/step - loss: 2.1299 - accuracy: 0.4826 - val_loss: 1.4548 - val_accuracy: 0.6667
Epoch 4/10
9/9 [==============================] - 0s 12ms/step - loss: 0.8238 - accuracy: 0.7951 - val_loss: 0.9495 - val_accuracy: 0.7083
Epoch 5/10
9/9 [==============================] - 0s 12ms/step - loss: 0.4160 - accuracy: 0.8819 - val_loss: 0.7734 - val_accuracy: 0.7917
Epoch 6/10
9/9 [==============================] - 0s 12ms/step - loss: 0.3971 - accuracy: 0.9097 - val_loss: 0.5509 - val_accuracy: 0.8056
Epoch 7/10
9/9 [==============================] - 0s 12ms/step - loss: 0.2015 - accuracy: 0.9306 - val_loss: 0.6047 - val_accuracy: 0.8194
Epoch 8/10
9/9 [=========

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets
